# MNIST Classifier

In this notebook you will create both, an mnist tabular dataset and a classifier.

## 1.- import the Operating System (os) module in python and any other library you need

In [4]:
import os
import zipfile
from PIL import Image
import numpy as np
import pandas as pd
from IPython.display import display

## Get the dataset

In [4]:
os.environ['KAGGLE_USERNAME'] = "anthonyemeka12"
os.environ['KAGGLE_KEY']      = "52a32a7dda673d7b0705657b185547c5"
!pip install --upgrade kaggle
!kaggle datasets download -d scolianni/mnistasjpg
DATA_PATH = "./mnistasjpg/"

#unzip
with zipfile.ZipFile('mnistasjpg.zip', 'r') as zip_ref:
    zip_ref.extractall(DATA_PATH)
os.remove('mnistasjpg.zip')


  0%|          | 0.00/68.4M [00:00<?, ?B/s]
  1%|▏         | 1.00M/68.4M [00:00<00:08, 8.64MB/s]
  4%|▍         | 3.00M/68.4M [00:00<00:05, 11.5MB/s]
  7%|▋         | 5.00M/68.4M [00:00<00:05, 11.7MB/s]
 10%|█         | 7.00M/68.4M [00:00<00:05, 11.7MB/s]
 13%|█▎        | 9.00M/68.4M [00:00<00:05, 11.8MB/s]
 16%|█▌        | 11.0M/68.4M [00:00<00:05, 11.8MB/s]
 19%|█▉        | 13.0M/68.4M [00:01<00:04, 11.8MB/s]
 22%|██▏       | 15.0M/68.4M [00:01<00:04, 11.8MB/s]
 25%|██▍       | 17.0M/68.4M [00:01<00:04, 11.8MB/s]
 28%|██▊       | 19.0M/68.4M [00:01<00:04, 11.8MB/s]
 31%|███       | 21.0M/68.4M [00:01<00:04, 11.8MB/s]
 34%|███▎      | 23.0M/68.4M [00:02<00:04, 11.3MB/s]
 37%|███▋      | 25.0M/68.4M [00:02<00:03, 12.0MB/s]
 39%|███▉      | 27.0M/68.4M [00:02<00:03, 12.0MB/s]
 42%|████▏     | 29.0M/68.4M [00:02<00:03, 11.7MB/s]
 45%|████▌     | 31.0M/68.4M [00:02<00:03, 12.0MB/s]
 48%|████▊     | 33.0M/68.4M [00:02<00:03, 11.6MB/s]
 51%|█████     | 35.0M/68.4M [00:03<00:02, 12.0MB/s]
 

## 2.- As you can see each class has its own folder (Do it only for train). 

    - Iterate folder by folder ( os.listdir() )
    - Inside each folder: 
        1.- Read the image
        2.- Reshape it into a flat array (784,)
        3.- Save the data into a pandas dataframe apending the column name as the class
    - Save the data into a CSV

    Note: if it takes to long try doing only 100 images per folder for the CSV.

In [5]:
def showImage(img,type='img'):
    if type == 'img':
        img = Image.open(img, 'r')
    else:
        dim = np.sqrt(img.reshape(1,-1).shape[0])
        img = img.reshape(dim,dim)
        img = Image.fromarray(img.astype(int))
        display(img)


In [8]:
train_path = 'mnistasjpg/trainingSet/trainingSet/'
test_path = 'mnistasjpg/testSet/testSet/'
train_folders = os.listdir(train_path)
test_folders = os.listdir(test_path)

for p in train_folders:
    p_img = os.listdir(train_path+p)

pth = train_path+p+'/'+p_img[0]
showImage(train_path+p+'/'+p_img[0])

In [3]:
from IPython.display import display # to display images

pil_im = Image.open(pth)
display(pil_im)

## 3.- Load the CSV

## 4.- Create a dictionary of models (No preprocessing needed, it has already been done).
    
    Include both, tree models and mult models.

## 5.- Using either cross validation or stratification find out which is the best model
    - Base your code on the previous two days examples

## Optional: Can you rotate an image?